**구글 텐서플로 라이브러리 임포트** <br>
CIFAR10 데이터셋은 컬러 이미지셋이므로 기존 neural network로는 분류가 쉽지 않다. 따라서 이미지 분류 전용의 CNN을 사용하려 한다. CNN 역시 tensorflow계열의 딥러닝 라이브러리이므로 필요한 파이썬 라이브러리를 임포트한다. 

In [12]:
from tensorflow.keras.datasets import cifar10
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation 
from sklearn.metrics import classification_report

**CIFAR10 데이터셋 다운로드** <br>
CIFAR10 데이터셋은 이미지 딥러닝 교육에 많이 사용되는 데이터셋으로 구글의 파이썬 명령 하나로 서버에서 다운로드 받을 수 있다. trainX는 트레이닝용 입력 이미지 데이터(비행기 이미지), trainY는 입력 이미지의 결과값 (비행기) 이다. 이미지 사이즈는 32x32이다. trainX.shpae 명령으로 사이즈를 확인할 수 있다. testX, testY는 검증용 데이터셋 이다. 

In [13]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()
# 입력 데이터인 trainX 사이즈를 확인하기 
print(trainX.shape)
# 실제 입력데이터 60000개 중 첫번째것의 실제 이미지 데이터를 확인하기 
print(trainX[0])
# 실제 출력데이터 60000개 중 첫번째것을 확인하기 
print(trainY[0])

(50000, 32, 32, 3)
[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]
[6]


**입력 이미지 데이터 노말라이징** <br>
입력 이미지 데이터는 정수의 배열로 데이터가 구성되어 있다. 이 데이터는 0 ~ 255까지의 숫자이다. 이 숫자를 0 ~ 1사이의 값으로 변경하는 것이 텐서플로에서 처리하기 쉽다.노말라이징은 입력 이미지 데이터인 trainX, testX가 대상이다. 

In [14]:
trainX = trainX.astype("float")/255
testX = testX.astype("float")/255
# trainX[0]의 정수 데이터들이 노말라이징 된 것을 확인한다.
print(trainX[0])

[[[0.23137255 0.24313725 0.24705882]
  [0.16862745 0.18039216 0.17647059]
  [0.19607843 0.18823529 0.16862745]
  ...
  [0.61960784 0.51764706 0.42352941]
  [0.59607843 0.49019608 0.4       ]
  [0.58039216 0.48627451 0.40392157]]

 [[0.0627451  0.07843137 0.07843137]
  [0.         0.         0.        ]
  [0.07058824 0.03137255 0.        ]
  ...
  [0.48235294 0.34509804 0.21568627]
  [0.46666667 0.3254902  0.19607843]
  [0.47843137 0.34117647 0.22352941]]

 [[0.09803922 0.09411765 0.08235294]
  [0.0627451  0.02745098 0.        ]
  [0.19215686 0.10588235 0.03137255]
  ...
  [0.4627451  0.32941176 0.19607843]
  [0.47058824 0.32941176 0.19607843]
  [0.42745098 0.28627451 0.16470588]]

 ...

 [[0.81568627 0.66666667 0.37647059]
  [0.78823529 0.6        0.13333333]
  [0.77647059 0.63137255 0.10196078]
  ...
  [0.62745098 0.52156863 0.2745098 ]
  [0.21960784 0.12156863 0.02745098]
  [0.20784314 0.13333333 0.07843137]]

 [[0.70588235 0.54509804 0.37647059]
  [0.67843137 0.48235294 0.16470588]


**출력 데이터셋 라벨 바이나리징** <br>
CIFAR10 출력 데이터셋 즉 라벨값은 0 ~ 9사이의 정수이다. 이 정수를 바이너리 벡터화 하는 것이 라벨바이나리징인데 이것을 실행한다. <br>
2 -> [0 0 1 0 0 0 0 0 0 0 0]

In [15]:
print(trainY[0])
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)
print(trainY[0]) # 라벨바이나리징 전후를 비교하기 

[6]
[0 0 0 0 0 0 1 0 0 0]


**라벨 데이터의 숫자 맵핑** <br>
CIFAR10 10개의 데이터의 이름을 숫자와 맵핑한다. 여기서는 참고로만 사용한다. 실제 코드에는 사용하지 않는다.   

In [16]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
	"dog", "frog", "horse", "ship", "truck"]

**CNN 모델을 구성하기전 준비** <br>
CNN 모델을 구성하기 위해서 사전 준비를 한다. 먼저 경사하강법을 실행하는 옵티마이저는 SGD로 정한다. 러닝레이트는 0,01로 한다.입력 이미지는 32x32x3 이다. 

In [17]:
opt = SGD(lr=0.01)

height = 32
width = 32
depth =3
model = Sequential()
inputShape = (height, width, depth)

if K.image_data_format() == "channels_first":
	inputShape = (depth, height, width)

모델의 구성 <br>
1개의 컨볼루션 레이어, 1개의 댄스레이어(기존 신경망 레이어)로 구성된 CNN 모델을 만든다. 컨볼루션 레이어의 액티베이션 함수는 relu, 댄스레이어의 액티베이션 함수는 softmax를 사용한다. 

In [18]:
model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("softmax"))

**모델의 구성** <br>
옵티마이저, 손실함수, 메트릭을 정해주고 CNN 신경망을 구성한다. 

In [19]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"] )


**모델 트레이닝** <br>
이제 CNN이 구성되었고, trainX, trainY 데이터셋을 이용해서 트레이닝 데이터를 진행한다. testX testY로 검증을 한다. 총 epochs는 40이다. 

In [20]:
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs =40)


Epoch 1/40
1563/1563 [==============================] - 44s 28ms/step - loss: 1.8082 - accuracy: 0.3645 - val_loss: 1.6400 - val_accuracy: 0.4285
Epoch 2/40
1563/1563 [==============================] - 43s 27ms/step - loss: 1.5555 - accuracy: 0.4613 - val_loss: 1.4934 - val_accuracy: 0.4769
Epoch 3/40
1563/1563 [==============================] - 42s 27ms/step - loss: 1.3933 - accuracy: 0.5142 - val_loss: 1.4272 - val_accuracy: 0.4922
Epoch 4/40
1563/1563 [==============================] - 45s 29ms/step - loss: 1.3078 - accuracy: 0.5444 - val_loss: 1.3102 - val_accuracy: 0.5351
Epoch 5/40
1563/1563 [==============================] - 44s 28ms/step - loss: 1.2602 - accuracy: 0.5606 - val_loss: 1.2988 - val_accuracy: 0.5416
Epoch 6/40
1563/1563 [==============================] - 43s 27ms/step - loss: 1.2215 - accuracy: 0.5744 - val_loss: 1.2854 - val_accuracy: 0.5420
Epoch 7/40
1563/1563 [==============================] - 43s 28ms/step - loss: 1.1896 - accuracy: 0.5871 - val_loss: 1.3897 -

**트레이닝 결과 확인하기** <br> 
트레이닝 한 결과를 확인한다. 실제로 CIFAR10 데이터셋은 CNN을 사용해도 예측률이 60$정도로 그리 좋지는 않다. 이것은 데이터셋의 해상도가 너무 낮고 데이터숫자가 많지 않아서 발생하는 현상이다. 

In [21]:
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))


313/313 [==============================] - 3s 9ms/step
              precision    recall  f1-score   support

    airplane       0.55      0.77      0.64      1000
  automobile       0.73      0.72      0.72      1000
        bird       0.55      0.35      0.43      1000
         cat       0.44      0.45      0.45      1000
        deer       0.57      0.51      0.54      1000
         dog       0.53      0.44      0.48      1000
        frog       0.71      0.71      0.71      1000
       horse       0.58      0.75      0.66      1000
        ship       0.75      0.65      0.70      1000
       truck       0.66      0.70      0.68      1000

    accuracy                           0.61     10000
   macro avg       0.61      0.61      0.60     10000
weighted avg       0.61      0.61      0.60     10000



**CNN 추론파일 저장하기** <br>
트레이닝된 추론파일을 다음에 사용하기 위해서 디스크에 저장한다. 

In [22]:
model.save('cifar10_cnn.h5')